In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import glob

In [2]:
pathlist = glob.glob('data/weibo/*/*/*.json')

In [3]:
import json
from pprint import pprint
dflist = []
for i in range(len(pathlist)):
    with open(pathlist[i]) as data_file:    
        data = json.load(data_file)
        d = {'text':data['text'],"userid":data['user']['id'],"province":data['user']['province'],"city":data['user']['city'],"created_at":data['user']['created_at'],"gender":data['user']['gender']}
        newdf = pd.DataFrame(data=d, index=['1'])
        dflist.append(newdf)
pprint(data)
df = pd.concat(dflist, ignore_index=True)

{u'attitudes_count': 0,
 u'base62_id': u'D1rzc8Gtz',
 u'biz_feature': 0,
 u'bmiddle_pic': u'http://ww4.sinaimg.cn/bmiddle/bb606f3agw1exihauyoabj20k00qo0u4.jpg',
 u'category': 31,
 u'comments_count': 0,
 u'created_at': u'Fri Oct 30 00:39:52 +0800 2015',
 u'darwin_tags': [],
 u'favorited': False,
 u'geo': None,
 u'id': 3903442982069905,
 u'idstr': u'3903442982069905',
 u'in_reply_to_screen_name': u'',
 u'in_reply_to_status_id': u'',
 u'in_reply_to_user_id': u'',
 u'mid': u'3903442982069905',
 u'mlevel': 0,
 u'original_pic': u'http://ww4.sinaimg.cn/large/bb606f3agw1exihauyoabj20k00qo0u4.jpg',
 u'pic_ids': [u'bb606f3agw1exihauyoabj20k00qo0u4',
              u'bb606f3agw1exihavc5pgj20k00qo3zy',
              u'bb606f3agw1exihav361qj20k00qo0v4',
              u'bb606f3agw1exihavhb0aj20qo0k0tbg',
              u'bb606f3agw1exihawscayj20v91jltki'],
 u'relation': None,
 u'reposts_count': 1,
 u'rid': u'0_0_0_2789506320176548201',
 u'source': u'<a href="http://weibo.com/" rel="nofollow">\u5fae\u5

In [4]:
df[0:10]

,city,created_at,gender,province,text,userid
0,6,Mon Sep 08 18:07:34 +0800 2014,f,31,hhhh关注你这么久终于明白了你一直说自己黑……[笑cry][笑cry][笑cry]（po主...,5283971921
1,2,Sun Apr 17 21:57:01 +0800 2011,f,400,回复@努力的芒果粒儿:浑身上下的黑有没有,2104528224
2,6,Mon Sep 08 18:07:34 +0800 2014,f,31,回复@Arielle_K:但是瘦！你说过黑就必要要瘦[嘻嘻]你狠瘦啦[心],5283971921
3,2,Mon Jul 28 15:14:47 +0800 2014,f,400,好看诶这个红色的！,5235009603
4,1000,Sat Jul 17 15:28:03 +0800 2010,f,100,据说这个小羊皮质量很好～,1777517177
5,4,Mon Mar 21 12:05:43 +0800 2011,f,31,光看到好身材了,2036999427
6,1,Wed Feb 03 11:37:05 +0800 2010,f,32,方便透露价格么,1689152700
7,2,Sun Apr 17 21:57:01 +0800 2011,f,400,回复@谷甜一:正价310镑,2104528224
8,1,Thu Aug 11 16:32:05 +0800 2011,f,33,MK等他打到RMB2000以下买就值了[偷笑],2302077230
9,2,Sun Apr 17 21:57:01 +0800 2011,f,400,回复@七月的july:嗯，摸起来就感觉很好,2104528224


# 1.Word Count 1:

## a: Count the number of posts mentioning each of the included brand names as well as the users who mentioned them. Hint: Pay attention to the variation of names. 

In [5]:
import re
# we can improve it by using fuzzy match in the future
def word_in_text(wordlist,text):
    wordlist = map(lambda x: x.lower(),wordlist)
    text = text.lower().replace(" ","")
    match = [re.search(word, text) for word in wordlist]
    for item in match:
        if item:
            return True
    return False

### number of posts mentioning Kate Spade

In [6]:
katespadeseries = df['text'].apply(lambda text: word_in_text([u"katespade",u"凯特·丝蓓","凯特丝蓓"],text))
katespadeseries.sum()

5611

### number of users who mentioned Kate Spade

In [7]:
len(df[katespadeseries].userid.unique())

2388

number of posts mentioning Michael Kors

In [8]:
michaelkorsseries = df['text'].apply(lambda text: word_in_text([u"michaelkors",u"邁可·寇斯","迈克高仕"],text))
michaelkorsseries.sum()

4610

number of users who mentioned Michael Kors

In [9]:
len(df[michaelkorsseries].userid.unique())

1761

##   b: List the top 10 users and locations (as province level in China and nation level worldwide) for total posts.

In [10]:
df["postcount"] = 1

### top 10 users for total posts (index:userid)

In [11]:
df.groupby("userid").sum().sort_values(by="postcount",ascending=False)[:10]

,postcount
userid,
3217535201,591
2187298830,271
3956831443,188
5689266274,155
5671523257,146
3737544502,143
1161106650,112
3481576475,98
5319321035,88


### top 10 locations as province level in china

check [province city code](http://open.weibo.com/wiki/%E7%9C%81%E4%BB%BD%E5%9F%8E%E5%B8%82%E7%BC%96%E7%A0%81%E8%A1%A8) for more details

In [12]:
df['province'] = map(lambda x: int(x),df.province)
df['city'] = map(lambda x: int(x),df.city)

In [13]:
df[df.province<99].groupby("province").sum().sort_values(by="postcount",ascending=False)["postcount"][0:10]

province
44    2180
31    1761
11    1681
33    1254
32    1109
51     936
42     586
81     573
35     462
37     447
Name: postcount, dtype: int64

### top 10 locations worldwide for total posts

1 => "美国", 2 => "英国", 3 => "法国", 4 => "俄罗斯", 5 => "加拿大", 
6 => "巴西", 7 => "澳大利亚", 8 => "印尼", 9 => "泰国", 10 => "马来西亚",
11 => "新加坡", 12 => "菲律宾", 13 => "越南", 14 => "印度", 15 => "日本", 16 => "其他"

check [province city code](http://open.weibo.com/wiki/%E7%9C%81%E4%BB%BD%E5%9F%8E%E5%B8%82%E7%BC%96%E7%A0%81%E8%A1%A8) for more details

china has the most posts:14901

In [14]:
chinapostnumber = len(df[df.province<99])
chinapostnumber

14901

In [15]:
df[df.province==400][df.city<=16].groupby("city").sum().sort_values(by="postcount",ascending=False)["postcount"][:10]

/Users/yutongpang/.virtualenvs/bombodaven27/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


city
1     4439
16     402
5      279
2      205
7      119
15     113
3       87
11      43
4       28
10      14
Name: postcount, dtype: int64

# 2.Word count II: 

## a: Find the date that has the highest number of posts mentioning each of the brands

In [16]:
df['created_at'] = pd.to_datetime(df.created_at)

In [17]:
df_date = df.copy()

In [18]:
import datetime
df_date['date'] = map(lambda x: datetime.datetime(x.year,x.month,x.day,0,0), df_date.created_at)

### For Michael Kors

In [19]:
michaelkorsseries = df_date['text'].apply(lambda text: word_in_text([u"michaelkors",u"邁可·寇斯","迈克高仕"],text))
df_date[michaelkorsseries].groupby("date").sum().sort_values(by="postcount",ascending=False)['postcount'][:10]

date
2015-08-29    146
2015-08-08    145
2013-08-22    123
2013-12-26    102
2011-09-22     70
2011-06-28     62
2011-06-15     52
2014-11-26     43
2013-09-01     41
2013-03-09     40
Name: postcount, dtype: int64

### For Kate Spade

In [20]:
katespadeseries = df_date['text'].apply(lambda text: word_in_text([u"katespade",u"凯特·丝蓓","凯特丝蓓"],text))
df_date[katespadeseries].groupby("date").sum().sort_values(by="postcount",ascending=False)['postcount'][:10]

date
2013-12-26    54
2013-05-20    46
2013-04-26    39
2011-07-22    39
2012-07-24    38
2011-05-30    35
2014-10-14    34
2015-01-18    33
2011-03-30    31
2011-05-09    29
Name: postcount, dtype: int64

## b: Find the peak hour with the most posts. 

In [21]:
df_hour = df.copy()

In [22]:
df_hour['hour'] = map(lambda x: x.hour, df_hour.created_at)

In [23]:
df_hour.groupby("hour").sum().sort_values(by="postcount",ascending=False)['postcount'][:10]

hour
5     1852
10    1647
6     1553
12    1535
14    1534
3     1499
13    1451
11    1419
2     1341
8     1323
Name: postcount, dtype: int64

# 3.Word count III: 
 Tokenize the comments and retrieve the top 10 mentioned Chinese terms associated with each brand from all texts. You may use 3rd party libraries such as Jieba to complete this task.

In [24]:
import jieba

## For Kate Spade

In [25]:
katespadeseries = df['text'].apply(lambda text: word_in_text([u"katespade",u"凯特·丝蓓","凯特丝蓓"],text))
kate_df = df[katespadeseries]

In [26]:
sentencelist = map(lambda text: list(jieba.cut(text,cut_all=True)),kate_df.text)
reducedwordlist = reduce(lambda x,y:x+y,sentencelist)

Building prefix dict from /Users/yutongpang/.virtualenvs/bombodaven27/lib/python2.7/site-packages/jieba/dict.txt ...
DEBUG:jieba:Building prefix dict from /Users/yutongpang/.virtualenvs/bombodaven27/lib/python2.7/site-packages/jieba/dict.txt ...
Loading model from cache /var/folders/v6/jvr0q_y16qz5zn12sqs76g9h0000gn/T/jieba.cache
DEBUG:jieba:Loading model from cache /var/folders/v6/jvr0q_y16qz5zn12sqs76g9h0000gn/T/jieba.cache
Loading model cost 0.336 seconds.
DEBUG:jieba:Loading model cost 0.336 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [27]:
from collections import Counter

In [28]:
sorted(Counter(reducedwordlist).items(),key=lambda wordtuple: wordtuple[1],reverse=True)[:30]

[(u'', 61699),
 (u'\u7684', 3460),
 (u'spade', 3173),
 (u'Kate', 2448),
 (u'\u5305', 2061),
 (u'kate', 2034),
 (u'Spade', 1598),
 (u'#', 1221),
 (u't', 1184),
 (u'cn', 1179),
 (u'http', 1159),
 (u'\u4e86', 1159),
 (u'\u7f8e\u56fd', 1155),
 (u'\u6298', 1095),
 (u'\u76f4', 916),
 (u'\u6298\u6263', 848),
 (u'\u5230\u624b', 769),
 (u'\u90ae', 748),
 (u'\u662f', 742),
 (u'\u6709', 709),
 (u'\u90fd', 651),
 (u'\u94b1\u5305', 631),
 (u'\u6b3e', 616),
 (u'\u6211', 607),
 (u'\u4ee3\u8d2d', 602),
 (u'\u8272', 541),
 (u'#kate', 533),
 (u'\u56fe', 529),
 (u'\u54e6', 506),
 (u'#Kate', 500)]

### The top 10 mentioned chinese terms which is meaningful is:

包,美国,折,直(邮),折扣,到手,邮,钱包,款,我,代购

## For Michael Kors

In [29]:
michaelkorsseries = df['text'].apply(lambda text: word_in_text([u"michaelkors",u"邁可·寇斯","迈克高仕"],text))
michael_df = df[michaelkorsseries]

In [30]:
sentencelist = map(lambda text: list(jieba.cut(text,cut_all=True)),michael_df.text)
reducedwordlist = reduce(lambda x,y:x+y,sentencelist)

In [31]:
sorted(Counter(reducedwordlist).items(),key=lambda wordtuple: wordtuple[1],reverse=True)[:30]

[(u'', 59636),
 (u'Michael', 2893),
 (u'\u7684', 2584),
 (u'Kors', 2086),
 (u'\u5305', 2020),
 (u'kors', 1394),
 (u'\u7f8e\u56fd', 1135),
 (u'#', 979),
 (u't', 866),
 (u'cn', 864),
 (u'http', 845),
 (u'\u4e2d\u53f7', 796),
 (u'\u76f4', 739),
 (u'\u4e86', 729),
 (u'\u662f', 720),
 (u'#Michael', 691),
 (u'\u8272', 681),
 (u'\u4ee3\u8d2d', 660),
 (u'\u90ae', 641),
 (u'\u6b3e', 590),
 (u'\u6709', 583),
 (u'\u552e\u4ef7', 569),
 (u'\u4e13\u67dc', 565),
 (u'MK', 562),
 (u'\u6b63\u54c1', 560),
 (u'\u5c3a\u5bf8', 539),
 (u'\u989c\u8272', 538),
 (u'\u6298\u6263', 529),
 (u'MICHAEL', 496),
 (u'\u90fd', 491)]

### The top 10 mentioned chinese terms which is meaningful is:

包,美国,中号,直(邮),色,代购,邮,款,有,售价

# 4. Sampling
Explain possible sampling bias through Weibo, such as gender bias, etc.

## 87.4% of posts of post by female user

In [32]:
len(df[df.gender=="f"])/float(len(df[df.gender=="f"])+len(df[df.gender=="m"]))

0.8743724847529353

gender, age, suburban or city, jobs, can be sources of sampling bias.
For example, female user tend to use weibo more than male user, and young people tend to use weibo more than old people.

# 5
What are some possible algorithms to identify users who showed interest in Michael Kors over Kate Spade? What are some data points that can be used to illustrate the algorithm's utility? Give a couple examples and discuss pros and cons. (You do not have to code here)

This can be a classifier problem in machine learning:

We first need to choose the predictor variables for the problem, including how many posts the user post related to eacd brand, how many reposts related to each brand, how many comments related to each brand, the age of the user, the gender of the user, city and provinces inforamtion...

After choosing the predictor variables, we can choose the machine learning algorithm (Decision tree, logistic regression, SVM, neural network) as the model.

Then we can divided the data set into traning data set, cross validaiton data set and test data set to train the model using cross validation method to choose the right regularization parameters.



# 6. What algorithms would you use to implement sentiment analysis? List pros & cons of each approach.

Sentiment analysis is a classifier problem in machine learning.

Before choosing the machine learning algorithm, we need to select the features (predictor variables), including a dictionary of positive and negative word, emotions sign, emoji...

After choosing the features, we can choose the machine learnign model:

1. Decision Tree:

    pros: interpretability, the decision tree does automatic feature selection, is faster to build and has less  
    parameters to tune.
    
    cons: It ussally has variance (overfitting issues). It can be extremely sensitive to small pertuabations in the       data.
2. Support Vector Machine:

    pros: It is effective in high dimensional spaces.
    
    cons: We need to chose kernel carefully, computation performance.
3. Neural Network:

    pros: simple to implement
    
    cons: black box, take long time to train

# 7. How would you be able to identify brand names in text?

We first need a dictionary includinng varitions of brand names. (The dictionary should including both chinese and eneglish version of the brand name, and we also need to remove the space between the word, and for the english word, we convert them into lower case.). We also need to take into account the typographical erros and new combinations of the name (fuzzy based word similarity function).

In [33]:
a = "michaelkors"
b = "maahaelkorsdwdasd"

In [34]:
import regex

In [35]:
r = regex.compile('('+a+'){e<=3}')
if r.match(b):
    print "match"
else:
    print "don't match"

match
